In [ ]:
import requests
import pandas as pd
import json

# Set API Key
apikey = "zYRJzdl7166yZe7IPFANB9NGjKqYWqqT8hMBqzjh"
url = "https://yfapi.net/v6/finance/quote"
headers = {'x-api-key': apikey}

## Fetching Stock Data
def fetch_stock_data():
    stocks = input("Enter at least two stock symbols separated by commas: ").upper()


    stock_list = [s.strip() for s in stocks.split(",") if s.strip()]
    if len(stock_list) < 2:
        print("Error: Please enter at least two stock symbols.")
        return

    querystring = {"symbols": ",".join(stock_list)}
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()


        if "quoteResponse" in data and "result" in data["quoteResponse"]:
            df = pd.DataFrame(data["quoteResponse"]["result"])


            df_task1 = df[['symbol', 'longName', 'regularMarketPrice']]


            df_task1 = df_task1.rename(columns={
                'symbol': 'Stock Ticker',
                'longName': 'Company Name',
                'regularMarketPrice': 'Current Market Price'
            })


            df_task1['Current Market Price'] = df_task1['Current Market Price'].apply(
                lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A"
            )


            print("\n✅ Task 1 Output:\n")
            print(df_task1.to_string(index=False))
        else:
            print("No stock data found.")
    else:
        print(f"Error {response.status_code}: {response.text}")

### Task 2.1
def get_stock_metrics(stocks):
    querystring = {"symbols": stocks}
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()

        if "quoteResponse" in data and "result" in data["quoteResponse"]:
            df = pd.DataFrame(data["quoteResponse"]["result"])


            available_columns = df.columns
            required_columns = ['symbol', 'longName', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow', 'returnOnAssets']
            existing_columns = [col for col in required_columns if col in available_columns]


            df_metrics = df[existing_columns]


            rename_map = {
                'symbol': 'Stock Ticker',
                'longName': 'Company Name',
                'fiftyTwoWeekHigh': '52-Week High',
                'fiftyTwoWeekLow': '52-Week Low',
                'returnOnAssets': 'ROA (%)'
            }
            df_metrics = df_metrics.rename(columns=rename_map)


            if '52-Week High' in df_metrics.columns:
                df_metrics['52-Week High'] = df_metrics['52-Week High'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
            if '52-Week Low' in df_metrics.columns:
                df_metrics['52-Week Low'] = df_metrics['52-Week Low'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
            if 'ROA (%)' in df_metrics.columns:
                df_metrics['ROA (%)'] = df_metrics['ROA (%)'].apply(lambda x: f"{x * 100:.2f}%" if pd.notna(x) else "N/A")

            print("\nTask 2.1 Output:\n")
            print(df_metrics.to_string(index=False))
        else:
            print("No stock data found.")
    else:
        print(f"Error {response.status_code}: {response.text}")

###  Task 2.2:
def get_trending_stocks():
    trending_url = "https://yfapi.net/v1/finance/trending"
    response = requests.get(trending_url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        if "finance" in data and "result" in data["finance"] and len(data["finance"]["result"]) > 0:
            trending_symbols = [item["symbol"] for item in data["finance"]["result"][0]["quotes"][:10]]


            querystring = {"symbols": ",".join(trending_symbols)}
            trending_response = requests.get(url, headers=headers, params=querystring)

            if trending_response.status_code == 200:
                trending_data = trending_response.json()

                if "quoteResponse" in trending_data and "result" in trending_data["quoteResponse"]:
                    df_trending = pd.DataFrame(trending_data["quoteResponse"]["result"])


                    df_trending = df_trending[['symbol', 'longName', 'regularMarketPrice', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow']]


                    df_trending = df_trending.rename(columns={
                        'symbol': 'Stock Ticker',
                        'longName': 'Company Name',
                        'regularMarketPrice': 'Current Price',
                        'fiftyTwoWeekHigh': '52-Week High',
                        'fiftyTwoWeekLow': '52-Week Low'
                    })


                    df_trending[['Current Price', '52-Week High', '52-Week Low']] = df_trending[['Current Price', '52-Week High', '52-Week Low']].applymap(
                        lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A"
                    )

                    print("\n Task 2.2 Output: Trending Stocks\n")
                    print(df_trending.to_string(index=False))
                else:
                    print("No trending stock data found.")
            else:
                print(f"Error {trending_response.status_code}: {trending_response.text}")
        else:
            print("No trending stock data available.")
    else:
        print(f"Error {response.status_code}: {response.text}")


fetch_stock_data()  # Run Task 1
stocks = input("\nEnter stock symbols for Task 2.1: ").upper()
if stocks:
    get_stock_metrics(stocks)
get_trending_stocks()  # Run Task 2.2


Enter at least two stock symbols separated by commas: AAPL, MSFT

✅ Task 1 Output:

Stock Ticker          Company Name Current Market Price
        AAPL            Apple Inc.              $240.36
        MSFT Microsoft Corporation              $399.73
